## Gradient Boosting 回帰モデルの SHAP による説明
自動車の価格を予測する回帰モデルを Gradient Boosting を用いて構築し、Interpret-community の SHAPベースの explainer を用いて説明 (グローバル、ローカル) を行います。

### 0. 事前準備
- Jupyter Kernel :  `rai-shap` を選択する。
    - [0-Setup.ipynb](./0-Setup.ipynb) の手順に従い構築しておくこと。

### 1. ライブラリ
必要な Python ライブラリをインポートします。

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# interpret-community 関連
from interpret.ext.blackbox import TabularExplainer
from interpret_community.common.constants import ModelTask

### 2. データ準備
自動車の価格に関するサンプルデータを Pandas DataFrame としてインポートします。

In [ ]:
import pandas as pd
df = pd.read_csv("../data/automobile.csv")
df = df.dropna()

In [ ]:
label = "price"

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# 学習データとテストデータに分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop([label], axis=1), df[label], test_size=0.5, random_state=1234)

### 3. 勾配ブースティングによるモデル構築

#### scikit learn パイプラインの作成

In [ ]:
# カテゴリー変数と数値変数のリスト (numerical, categorical)
categorical = []
for col, value in X_test.iteritems():
    if value.dtype == 'object':
        categorical.append(col)
        
numerical = X_test.columns.difference(categorical).tolist()

In [ ]:
# 特徴量の一覧
features = X_train.columns.tolist()

In [ ]:
from sklearn.compose import ColumnTransformer

# 前処理パイプライン
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
    ])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])


In [ ]:
# モデル学習パイプライン
reg = GradientBoostingRegressor(n_estimators=20, max_depth=10,
                                learning_rate=0.1, loss='huber',
                                random_state=1234)

In [ ]:
# 全体のパイプライン
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('regression', reg)])

#### モデル学習　

In [ ]:
model = clf.fit(X_train, y_train)

### 4. モデル説明 (SHAP Tabular Explainer)
interpret-community の `TabularExplainer` を用いて explainer を生成します。TabularExplainer は最適な SHAP Explainer を自動で選択します。

In [ ]:
explainer = TabularExplainer(model.steps[-1][1], 
                             initialization_examples=X_train,
                             features=features,
                             transformations=transformations,
                             model_task = ModelTask.Regression)

#### グローバル (大局的) 解釈

In [ ]:
# データの母集団を引数に渡す。テストデータ X_test でも可。
global_explanation = explainer.explain_global(X_train)

In [ ]:
# 変数重要度 (降順)
global_explanation.get_feature_importance_dict()

#### ローカル (局所的) 解釈

In [ ]:
# X_test の 3 番目のデータについての予測値を解釈
num = 3
local_explanation_filtered = explainer.explain_local(X_test[num:num+1])

In [ ]:
sorted_local_importance_names = local_explanation_filtered.get_ranked_local_names()
sorted_local_importance_values = local_explanation_filtered.get_ranked_local_values()

In [ ]:
# 変数重要度 (降順)
print(sorted_local_importance_names, sorted_local_importance_values)

#### ダッシュボード可視化

In [ ]:
from raiwidgets import ExplanationDashboard
ExplanationDashboard(global_explanation, model, dataset=X_train, true_y=y_train.to_numpy())

In [ ]:
ExplanationDashboard(explainer.explain_local(X_test), 
                     model, dataset=X_test, true_y=y_test.to_numpy())

### 5. モデル誤差分析 (Error Analysis)
[Error Analysis](https://erroranalysis.ai/) を用いてモデルの誤差を分析し、特に誤差が大きいコホートを特定します。

In [ ]:
from raiwidgets import ErrorAnalysisDashboard

In [ ]:
local_explanation = explainer.explain_local(X_test)

In [ ]:
ErrorAnalysisDashboard(local_explanation, model,
                       dataset=X_test, 
                       true_y=y_test.to_numpy(),
                       model_task="regression",
                       categorical_features=categorical,
                       true_y_dataset=y_test.to_numpy(),
                       max_depth=10)